In [2]:
import pandas as pd
import plotly.graph_objects as go
import os
import statsmodels.api as sm
from tabulate import tabulate

# Set the file paths
myfitnesspal_path = '/Users/ngirmay/Documents/GitHub/ironman_retrospective/IronMan_2023/myfitnesspal/'
base_path = '/Users/ngirmay/Documents/GitHub/ironman_retrospective/apple_health/health_data_exported/'
body_mass_file = os.path.join(base_path, 'HKQuantityTypeIdentifierBodyMass_2024-04-99_18-30-56_SimpleHealthExportCSV.csv')

# Read the CSV files
nutrition_df = pd.read_csv(os.path.join(myfitnesspal_path, 'Nutrition-Summary-2016-03-13-to-2024-01-19.csv'))
exercise_df = pd.read_csv(os.path.join(myfitnesspal_path, 'Exercise-Summary-2016-03-13-to-2024-01-19.csv'))
body_mass_df = pd.read_csv(body_mass_file, skiprows=1)  # Skip the first row as it contains 'sep=,'

# Convert date columns to datetime and ensure UTC timezone
nutrition_df['Date'] = pd.to_datetime(nutrition_df['Date'], utc=True)
exercise_df['Date'] = pd.to_datetime(exercise_df['Date'], utc=True)
body_mass_df['startDate'] = pd.to_datetime(body_mass_df['startDate'], utc=True)

# Rename columns and select relevant ones for body mass data
body_mass_df = body_mass_df[['startDate', 'value']].rename(columns={'startDate': 'Date', 'value': 'Weight'})

# Group nutrition data by date and sum the values
nutrition_daily = nutrition_df.groupby('Date').agg({
    'Calories': 'sum',
    'Protein (g)': 'sum',
    'Carbohydrates (g)': 'sum',
    'Fat (g)': 'sum'
}).reset_index()

# Merge nutrition data with body mass data
combined_df = pd.merge(body_mass_df, nutrition_daily, on='Date', how='outer')

# Merge exercise data
exercise_daily = exercise_df.groupby('Date').agg({
    'Exercise Calories': 'sum',
    'Exercise Minutes': 'sum'
}).reset_index()

combined_df = pd.merge(combined_df, exercise_daily, on='Date', how='outer')

# Sort by date
combined_df = combined_df.sort_values('Date')

# Fill NaN values with forward fill method
combined_df = combined_df.ffill()

# Calculate net calories (intake - exercise)
combined_df['Net Calories'] = combined_df['Calories'] - combined_df['Exercise Calories'].fillna(0)

# Filter data for Ironman training period
start_date = '2022-12-01'
end_date = '2023-07-23'
ironman_df = combined_df[(combined_df['Date'] >= start_date) & (combined_df['Date'] <= end_date)]

# Function to save and display plotly figure
def save_and_display_plot(fig, filename):
    html_path = os.path.join(myfitnesspal_path, filename)
    fig.write_html(html_path)
    print(f"Plot saved as {html_path}")
    fig.show()

# 1. Weight and Net Calories Over Time
fig = go.Figure()

# Add Weight trace
fig.add_trace(go.Scatter(
    x=ironman_df['Date'],
    y=ironman_df['Weight'],
    mode='lines+markers',
    name='Weight',
    line=dict(color='#457b9d')
))

# Add Net Calories trace on secondary y-axis
fig.add_trace(go.Scatter(
    x=ironman_df['Date'],
    y=ironman_df['Net Calories'],
    mode='lines+markers',
    name='Net Calories',
    line=dict(color='#e63946'),
    yaxis='y2'
))

# Update layout
fig.update_layout(
    title="Weight and Net Calories During Ironman Training",
    xaxis=dict(title='Date'),
    yaxis=dict(
        title='Weight (lbs)',
        titlefont=dict(color='#457b9d'),
        tickfont=dict(color='#457b9d')
    ),
    yaxis2=dict(
        title='Net Calories',
        titlefont=dict(color='#e63946'),
        tickfont=dict(color='#e63946'),
        overlaying='y',
        side='right'
    ),
    legend=dict(x=1.1, y=1),
    plot_bgcolor='#f1faee'
)

save_and_display_plot(fig, 'ironman_weight_and_calories_over_time.html')

# 2. Macronutrient Distribution
color_mapping = {
    'Protein': '#e63946',
    'Carbohydrates': '#a8dadc',
    'Fat': '#457b9d'
}

macronutrient_data = [
    ('Protein', ironman_df['Protein (g)'].mean()),
    ('Carbohydrates', ironman_df['Carbohydrates (g)'].mean()),
    ('Fat', ironman_df['Fat (g)'].mean())
]

fig = go.Figure(data=[go.Pie(
    labels=[item[0] for item in macronutrient_data],
    values=[item[1] for item in macronutrient_data],
    marker=dict(colors=[color_mapping[item[0]] for item in macronutrient_data]),
    textinfo='label+percent',
    hole=.3
)])

fig.update_layout(
    title="Average Macronutrient Distribution During Ironman Training",
    plot_bgcolor='#f1faee'
)

save_and_display_plot(fig, 'ironman_macronutrient_distribution.html')

# Prepare summary statistics
summary_stats = pd.DataFrame({
    'Metric': ['Total days', 'Average daily calorie intake', 'Average daily net calories',
               'Average daily exercise calories', 'Average daily exercise minutes',
               'Starting weight', 'Ending weight', 'Total weight change',
               'Average daily protein intake', 'Average daily carbohydrate intake',
               'Average daily fat intake'],
    'Value': [
        len(ironman_df),
        f"{ironman_df['Calories'].mean():.2f}",
        f"{ironman_df['Net Calories'].mean():.2f}",
        f"{ironman_df['Exercise Calories'].mean():.2f}",
        f"{ironman_df['Exercise Minutes'].mean():.2f}",
        f"{ironman_df['Weight'].iloc[0]:.2f} lbs",
        f"{ironman_df['Weight'].iloc[-1]:.2f} lbs",
        f"{ironman_df['Weight'].iloc[-1] - ironman_df['Weight'].iloc[0]:.2f} lbs",
        f"{ironman_df['Protein (g)'].mean():.2f} g",
        f"{ironman_df['Carbohydrates (g)'].mean():.2f} g",
        f"{ironman_df['Fat (g)'].mean():.2f} g"
    ]
})

exercise_days = ironman_df['Exercise Calories'].notna().sum()
total_days = len(ironman_df)
exercise_percentage = (exercise_days / total_days) * 100

# Add the exercise percentage to the summary stats using concat instead of append
summary_stats = pd.concat([summary_stats, pd.DataFrame({
    'Metric': ['Percentage of days with recorded exercise'],
    'Value': [f"{exercise_percentage:.2f}%"]
})], ignore_index=True)

print("\nIronman Training Period Analysis (Dec 1, 2022 - Jul 23, 2023):")
print(tabulate(summary_stats, headers='keys', tablefmt='pretty', showindex=False))

# Calculate correlations
correlations = ironman_df[['Weight', 'Net Calories', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Exercise Calories']].corr()
correlation_table = correlations['Weight'].sort_values(ascending=False).reset_index()
correlation_table.columns = ['Variable', 'Correlation with Weight']

print("\nCorrelations with Weight:")
print(tabulate(correlation_table, headers='keys', tablefmt='pretty', showindex=False, floatfmt='.4f'))

# Statistical Analysis: Multiple Linear Regression
print("\nMultiple Linear Regression Analysis:")

# Prepare the data
X = ironman_df[['Net Calories', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Exercise Calories']]
y = ironman_df['Weight']

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Prepare regression results table
regression_results = pd.DataFrame({
    'Variable': model.model.exog_names,
    'Coefficient': model.params,
    'P-value': model.pvalues,
    'Std Error': model.bse
})

print("\nRegression Results:")
print(tabulate(regression_results, headers='keys', tablefmt='pretty', showindex=False, floatfmt='.4f'))

print(f"\nR-squared: {model.rsquared:.4f}")
print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")

print("\nInterpretation of the Multiple Linear Regression Results:")
print("1. R-squared value indicates the proportion of the variance in Weight that is predictable from the independent variables.")
print("2. Variables with p-values < 0.05 are considered statistically significant.")
print("3. Coefficients represent the change in Weight for a one-unit change in the respective variable, holding other variables constant.")
print("\nNote: This analysis assumes a linear relationship and independence of observations, which may not hold perfectly for time-series data.")

Plot saved as /Users/ngirmay/Documents/GitHub/ironman_retrospective/IronMan_2023/myfitnesspal/ironman_weight_and_calories_over_time.html


Plot saved as /Users/ngirmay/Documents/GitHub/ironman_retrospective/IronMan_2023/myfitnesspal/ironman_macronutrient_distribution.html



Ironman Training Period Analysis (Dec 1, 2022 - Jul 23, 2023):
+-------------------------------------------+------------+
|                  Metric                   |   Value    |
+-------------------------------------------+------------+
|                Total days                 |    111     |
|       Average daily calorie intake        |  2191.40   |
|        Average daily net calories         |  2046.40   |
|      Average daily exercise calories      |   145.00   |
|      Average daily exercise minutes       |   45.00    |
|              Starting weight              | 167.20 lbs |
|               Ending weight               | 156.60 lbs |
|            Total weight change            | -10.60 lbs |
|       Average daily protein intake        |  152.17 g  |
|     Average daily carbohydrate intake     |  269.74 g  |
|         Average daily fat intake          |  56.55 g   |
| Percentage of days with recorded exercise |  100.00%   |
+-------------------------------------------+------

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from scipy.stats import binomtest
import numpy as np
import os

# Create a directory for saving visualizations
output_dir = '/Users/ngirmay/Documents/GitHub/ironman_retrospective/IronMan_2023/diet/'
os.makedirs(output_dir, exist_ok=True)

# Color mapping for the zones
color_mapping = {
    'Zone 1 (50%-60%)': '#e63946',
    'Zone 2 (60%-70%)': '#a8dadc',
    'Zone 3 (70%-80%)': '#90EE90',
    'Zone 4 (80%-90%)': '#457b9d',
    'Zone 5 (90%-100%)': '#1d3557'
}

# Load SNP data from personal file
def load_snp_data(file_path):
    metadata = {}
    with open(file_path, 'r') as f:
        for i in range(7):
            line = f.readline().strip()
            key, value = line[2:].split(': ')
            metadata[key] = value
        header = f.readline().strip().lstrip('# ').split('\t')
    
    snp_data = pd.read_csv(file_path, sep='\t', skiprows=8, names=header, 
                           dtype={'rsid': str, 'chromosome': str, 'position': int, 'genotype': str},
                           low_memory=False)
    
    return metadata, snp_data

# Example: Load your SNP data
snp_file_path = '/Users/ngirmay/Documents/GitHub/ironman_retrospective/IronMan_2023/diet/selfdecode.txt'
metadata, snp_data = load_snp_data(snp_file_path)

# 1. Circular SNP Density Plot (holistic plot like Circos)
def plot_circular_snp_density(snp_data, output_path):
    snp_data['density'] = snp_data.groupby('chromosome')['position'].transform('count')

    theta = np.linspace(0, 2 * np.pi, len(snp_data))
    r = snp_data['density']

    fig = go.Figure()

    # Add SNP density per chromosome
    fig.add_trace(go.Scatterpolar(r=r, theta=theta, mode='markers',
                                  marker=dict(color=r, colorscale='Viridis', size=5)))
    
    fig.update_layout(title='Circular SNP Density Plot', polar=dict(radialaxis=dict(visible=True)))
    fig.write_html(output_path)

plot_circular_snp_density(snp_data, os.path.join(output_dir, 'circular_snp_density.html'))

# 2. SNP Distance Distribution (Log Scale and Zoom)
def snp_distance_distribution(snp_data, output_path):
    snp_data = snp_data.sort_values(['chromosome', 'position'])
    snp_data['distance'] = snp_data.groupby('chromosome')['position'].diff().fillna(0)
    
    # Filter to remove small distances that dominate the plot
    snp_data_filtered = snp_data[snp_data['distance'] > 1000]
    
    # Log scale to emphasize differences in larger distances
    fig = px.histogram(snp_data_filtered, x='distance', title='Distribution of SNP Distances by Chromosome (Log Scale)',
                       color_discrete_sequence=[color_mapping['Zone 1 (50%-60%)']], log_x=True, nbins=100)
    fig.update_layout(xaxis_title="Distance between SNPs (Log Scale)", yaxis_title="Count")
    fig.write_html(output_path)

snp_distance_distribution(snp_data, os.path.join(output_dir, 'snp_distance_distribution.html'))

# 3. Enhanced DNA Helix Plot with Minor Allele Frequency (MAF)
def calculate_maf(snp_data):
    allele_counts = snp_data['genotype'].apply(lambda g: list(g)).explode().value_counts()
    total_alleles = allele_counts.sum()
    maf = allele_counts.min() / total_alleles
    return maf

# Apply the function
maf_values = calculate_maf(snp_data)

def dna_helix_plot(snp_data, output_path):
    # Filter valid genotypes where the string length is at least 2
    valid_snp_data = snp_data[snp_data['genotype'].apply(lambda g: len(g) >= 2)]
    
    # Generate points for the helix
    theta = np.linspace(0, 4 * np.pi, len(valid_snp_data))
    z = np.linspace(0, 20, len(valid_snp_data))
    x = np.sin(theta)
    y = np.cos(theta)
    
    # Assign colors based on MAF
    maf_colors = np.random.rand(len(valid_snp_data))  # Use actual MAF calculation if available
    
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', 
                                       marker=dict(size=5, color=maf_colors, opacity=0.8, colorscale='Viridis'))])
    fig.update_layout(title='DNA Helix Visualization with SNPs (Colored by MAF)',
                      scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Position'))
    fig.write_html(output_path)

dna_helix_plot(snp_data, os.path.join(output_dir, 'dna_helix_plot.html'))

# 4. PCA of Genotypes
def plot_pca_genotypes(snp_data, output_path):
    snp_data['encoded_genotype'] = LabelEncoder().fit_transform(snp_data['genotype'].astype(str))
    snp_data['encoded_chromosome'] = LabelEncoder().fit_transform(snp_data['chromosome'].astype(str))

    features = snp_data[['encoded_genotype', 'encoded_chromosome', 'position']]
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(features)
    
    snp_data['PCA1'] = pca_result[:, 0]
    snp_data['PCA2'] = pca_result[:, 1]
    
    fig = px.scatter(snp_data, x='PCA1', y='PCA2', color='chromosome', 
                     title='PCA of SNP Genotypes',
                     color_continuous_scale=[color_mapping['Zone 1 (50%-60%)'],
                                             color_mapping['Zone 2 (60%-70%)'], 
                                             color_mapping['Zone 3 (70%-80%)'], 
                                             color_mapping['Zone 4 (80%-90%)'], 
                                             color_mapping['Zone 5 (90%-100%)']])
    fig.write_html(output_path)

plot_pca_genotypes(snp_data, os.path.join(output_dir, 'pca_genotypes.html'))

# 5. Homozygosity vs Heterozygosity
def homozygosity_heterozygosity_analysis(snp_data, output_path):
    def classify_genotype(g):
        if len(g) < 2:
            return 'Unknown'
        return 'Homozygous' if g[0] == g[1] else 'Heterozygous'

    snp_data['homozygosity'] = snp_data['genotype'].apply(classify_genotype)
    homozygous_count = snp_data[snp_data['homozygosity'] == 'Homozygous'].shape[0]
    heterozygous_count = snp_data[snp_data['homozygosity'] == 'Heterozygous'].shape[0]
    unknown_count = snp_data[snp_data['homozygosity'] == 'Unknown'].shape[0]

    fig = go.Figure(data=[go.Pie(labels=['Homozygous', 'Heterozygous', 'Unknown'],
                                 values=[homozygous_count, heterozygous_count, unknown_count],
                                 marker=dict(colors=[color_mapping['Zone 2 (60%-70%)'], 
                                                     color_mapping['Zone 3 (70%-80%)'], 
                                                     color_mapping['Zone 4 (80%-90%)']])),
                      ])
    fig.update_layout(title='Homozygous vs Heterozygous SNPs')
    fig.write_html(output_path)

homozygosity_heterozygosity_analysis(snp_data, os.path.join(output_dir, 'homozygosity_heterozygosity.html'))

# 6. Genotype Heatmap by Chromosome
def plot_genotype_heatmap(snp_data, output_path):
    snp_counts = snp_data.groupby(['chromosome', 'genotype']).size().unstack(fill_value=0)
    fig = px.imshow(snp_counts, title='Genotype Heatmap by Chromosome', 
                    labels={'color': 'SNP Count'},
                    color_continuous_scale=[color_mapping['Zone 1 (50%-60%)'], color_mapping['Zone 2 (60%-70%)'], color_mapping['Zone 3 (70%-80%)']])
    fig.write_html(output_path)

plot_genotype_heatmap(snp_data, os.path.join(output_dir, 'genotype_heatmap.html'))

# 7. Runs of Homozygosity (ROH)
def runs_of_homozygosity(snp_data, output_path):
    def is_homozygous(genotype):
        if len(genotype) < 2:
            return None  # Return None for invalid or missing genotypes
        return genotype[0] == genotype[1]

    # Apply the function to classify homozygous SNPs
    snp_data['is_homozygous'] = snp_data['genotype'].apply(is_homozygous)
    
    # Remove rows where 'is_homozygous' could not be determined (i.e., None)
    snp_data_clean = snp_data.dropna(subset=['is_homozygous'])
    
    # Calculate runs of homozygosity
    roh_data = snp_data_clean[snp_data_clean['is_homozygous']].groupby('chromosome')['position'].apply(lambda x: x.max() - x.min()).reset_index()
    roh_data.columns = ['chromosome', 'roh_length']
    
    # Plot the runs of homozygosity
    fig = px.bar(roh_data, x='chromosome', y='roh_length', title='Runs of Homozygosity (ROH) per Chromosome',
                 color_discrete_sequence=[color_mapping['Zone 4 (80%-90%)']])
    fig.write_html(output_path)

runs_of_homozygosity(snp_data, os.path.join(output_dir, 'runs_of_homozygosity.html'))

# 8. Homozygous/Heterozygous Density Per Chromosome
def plot_homozygous_heterozygous_density(snp_data, output_path):
    def classify_homozygosity(g):
        if len(g) < 2:
            return None  # Handle cases with missing or malformed genotype data
        return 1 if g[0] == g[1] else 0

    snp_data['homozygous'] = snp_data['genotype'].apply(classify_homozygosity)
    
    # Drop rows where homozygosity could not be determined (i.e., None)
    snp_data_clean = snp_data.dropna(subset=['homozygous'])
    
    density_data = snp_data_clean.groupby('chromosome')['homozygous'].mean().reset_index()
    density_data['heterozygous'] = 1 - density_data['homozygous']
    
    fig = go.Figure(data=[
        go.Bar(name='Homozygous', x=density_data['chromosome'], y=density_data['homozygous'],
               marker_color=color_mapping['Zone 2 (60%-70%)']),
        go.Bar(name='Heterozygous', x=density_data['chromosome'], y=density_data['heterozygous'],
               marker_color=color_mapping['Zone 3 (70%-80%)'])
    ])
    
    fig.update_layout(barmode='stack', title='Homozygous/Heterozygous Density Per Chromosome')
    fig.write_html(output_path)

plot_homozygous_heterozygous_density(snp_data, os.path.join(output_dir, 'homozygous_heterozygous_density.html'))
